# Day 03: Online Learning for Trading Models

## Week 23: Production ML

---

## Learning Objectives

1. Understand online learning vs batch learning paradigms
2. Implement incremental learning algorithms for streaming financial data
3. Detect and handle concept drift in trading models
4. Build adaptive models that learn from new market regimes
5. Apply online learning libraries (River, scikit-learn) to trading strategies

---

## Why Online Learning for Trading?

Financial markets are **non-stationary** - statistical properties change over time due to:
- Market regime shifts (bull/bear markets)
- Structural changes (regulations, new instruments)
- Alpha decay (strategies lose edge as others discover them)
- Changing volatility regimes

**Online learning** allows models to:
- Adapt to new patterns without full retraining
- Process streaming data efficiently
- Maintain performance during regime changes
- Reduce computational costs vs frequent batch retraining

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Online learning
from river import linear_model, preprocessing, metrics, drift, ensemble
from river import stream, optim, compose

# Scikit-learn incremental learning
from sklearn.linear_model import SGDClassifier, SGDRegressor, PassiveAggressiveClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error

# Data
import yfinance as yf

# Visualization
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries loaded successfully!")

---

## 1. Online Learning Fundamentals

### Batch vs Online Learning

| Aspect | Batch Learning | Online Learning |
|--------|---------------|----------------|
| Data | All at once | One sample at a time |
| Memory | High (store all data) | Low (constant) |
| Adaptation | Requires retraining | Continuous updates |
| Computation | Heavy periodic jobs | Light continuous |
| Use Case | Static environments | Streaming/changing data |

In [ ]:
# Fetch historical data for experiments
symbols = ['SPY', 'QQQ', 'IWM']
start_date = '2020-01-01'
end_date = '2024-12-31'

data = {}
for symbol in symbols:
    df = yf.download(symbol, start=start_date, end=end_date, progress=False)
    df.columns = df.columns.droplevel(1) if isinstance(df.columns, pd.MultiIndex) else df.columns
    data[symbol] = df

# Use SPY as primary dataset
df = data['SPY'].copy()
print(f"Data shape: {df.shape}")
print(f"Date range: {df.index[0]} to {df.index[-1]}")
df.head()

In [ ]:
def create_features(df, lookbacks=[5, 10, 20, 60]):
    """
    Create features for online learning trading model.
    Features designed to capture momentum, volatility, and mean-reversion.
    """
    features = pd.DataFrame(index=df.index)
    
    # Returns
    features['returns'] = df['Close'].pct_change()
    
    # Momentum features
    for lb in lookbacks:
        features[f'momentum_{lb}'] = df['Close'].pct_change(lb)
        features[f'volatility_{lb}'] = features['returns'].rolling(lb).std()
        features[f'rsi_{lb}'] = compute_rsi(df['Close'], lb)
    
    # Volume features
    features['volume_change'] = df['Volume'].pct_change()
    features['volume_ma_ratio'] = df['Volume'] / df['Volume'].rolling(20).mean()
    
    # Price position
    features['high_low_range'] = (df['High'] - df['Low']) / df['Close']
    features['close_position'] = (df['Close'] - df['Low']) / (df['High'] - df['Low'])
    
    # Moving average relationships
    features['ma_20_ratio'] = df['Close'] / df['Close'].rolling(20).mean() - 1
    features['ma_50_ratio'] = df['Close'] / df['Close'].rolling(50).mean() - 1
    
    # Target: Next day return direction (1 = up, 0 = down)
    features['target'] = (features['returns'].shift(-1) > 0).astype(int)
    features['target_return'] = features['returns'].shift(-1)
    
    return features.dropna()


def compute_rsi(series, period):
    """Compute Relative Strength Index."""
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))


# Create feature dataset
features_df = create_features(df)
print(f"Feature dataset shape: {features_df.shape}")
features_df.head()

---

## 2. Online Learning with River

[River](https://riverml.xyz/) is a Python library for online machine learning. It's designed for:
- Single-pass learning (each observation seen once)
- Constant memory usage
- Real-time predictions and updates

In [ ]:
# Prepare data for River (dict format)
feature_cols = [col for col in features_df.columns 
                if col not in ['target', 'target_return', 'returns']]

X = features_df[feature_cols]
y = features_df['target']

print(f"Features: {feature_cols}")
print(f"Number of samples: {len(X)}")

In [ ]:
# Build River online learning pipeline
model_river = compose.Pipeline(
    preprocessing.StandardScaler(),
    linear_model.LogisticRegression(optimizer=optim.SGD(lr=0.01))
)

# Metrics to track
metric = metrics.Accuracy()
rolling_metric = metrics.Rolling(metrics.Accuracy(), window_size=100)

# Store results for analysis
results = {
    'date': [],
    'prediction': [],
    'actual': [],
    'accuracy': [],
    'rolling_accuracy': []
}

# Online learning loop - process one sample at a time
for i, (idx, row) in enumerate(X.iterrows()):
    x_dict = row.to_dict()  # River expects dict format
    y_true = int(y.loc[idx])
    
    # Predict BEFORE learning (prequential evaluation)
    y_pred = model_river.predict_one(x_dict)
    
    # Update metrics
    if y_pred is not None:
        metric.update(y_true, y_pred)
        rolling_metric.update(y_true, y_pred)
        
        results['date'].append(idx)
        results['prediction'].append(y_pred)
        results['actual'].append(y_true)
        results['accuracy'].append(metric.get())
        results['rolling_accuracy'].append(rolling_metric.get())
    
    # Learn from this sample
    model_river.learn_one(x_dict, y_true)

print(f"Final Accuracy: {metric.get():.4f}")
results_df = pd.DataFrame(results)
results_df.set_index('date', inplace=True)

In [ ]:
# Visualize online learning performance over time
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Cumulative accuracy
axes[0].plot(results_df.index, results_df['accuracy'], label='Cumulative Accuracy', alpha=0.8)
axes[0].axhline(y=0.5, color='r', linestyle='--', label='Random Baseline')
axes[0].set_ylabel('Accuracy')
axes[0].set_title('Online Learning: Cumulative Accuracy Over Time')
axes[0].legend()

# Rolling accuracy (100-day window)
axes[1].plot(results_df.index, results_df['rolling_accuracy'], 
             label='100-Day Rolling Accuracy', color='green', alpha=0.8)
axes[1].axhline(y=0.5, color='r', linestyle='--', label='Random Baseline')
axes[1].set_ylabel('Rolling Accuracy')
axes[1].set_xlabel('Date')
axes[1].set_title('Online Learning: Rolling Accuracy (Window=100)')
axes[1].legend()

plt.tight_layout()
plt.show()

---

## 3. Concept Drift Detection

**Concept drift** occurs when the statistical properties of the target variable change over time. In trading:
- Market regime changes
- Volatility shifts
- Correlation breakdowns

### Common Drift Detectors:
1. **ADWIN (Adaptive Windowing)**: Automatically adjusts window size
2. **DDM (Drift Detection Method)**: Monitors error rate statistics
3. **Page-Hinkley**: Detects changes in mean of Gaussian signals

In [ ]:
# Implement concept drift detection
from river.drift import ADWIN, DDM, PageHinkley

# Create drift detectors
adwin = ADWIN(delta=0.002)
ddm = DDM()
ph = PageHinkley()

# Track drift points
drift_points = {
    'ADWIN': [],
    'DDM': [],
    'PageHinkley': []
}

# Simulate streaming with drift detection
model_drift = compose.Pipeline(
    preprocessing.StandardScaler(),
    linear_model.LogisticRegression(optimizer=optim.SGD(lr=0.01))
)

errors = []

for i, (idx, row) in enumerate(X.iterrows()):
    x_dict = row.to_dict()
    y_true = int(y.loc[idx])
    
    y_pred = model_drift.predict_one(x_dict)
    
    if y_pred is not None:
        error = int(y_pred != y_true)
        errors.append(error)
        
        # Update drift detectors with error signal
        adwin.update(error)
        ddm.update(error)
        ph.update(error)
        
        # Check for drift
        if adwin.drift_detected:
            drift_points['ADWIN'].append(idx)
        if ddm.drift_detected:
            drift_points['DDM'].append(idx)
        if ph.drift_detected:
            drift_points['PageHinkley'].append(idx)
    
    model_drift.learn_one(x_dict, y_true)

print("Drift Detection Summary:")
for detector, points in drift_points.items():
    print(f"  {detector}: {len(points)} drift points detected")

In [ ]:
# Visualize drift points with SPY price
fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# SPY price with drift points
axes[0].plot(df.index, df['Close'], label='SPY Close', alpha=0.8)

colors = {'ADWIN': 'red', 'DDM': 'orange', 'PageHinkley': 'purple'}
for detector, points in drift_points.items():
    if points:
        for point in points[:50]:  # Limit for visibility
            axes[0].axvline(x=point, color=colors[detector], alpha=0.3, linewidth=0.5)

axes[0].set_ylabel('Price')
axes[0].set_title('SPY Price with Detected Drift Points')
axes[0].legend()

# Rolling error rate
error_series = pd.Series(errors, index=results_df.index)
rolling_error = error_series.rolling(50).mean()
axes[1].plot(rolling_error.index, rolling_error, label='50-Day Rolling Error Rate', color='red')
axes[1].axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
axes[1].set_ylabel('Error Rate')
axes[1].set_xlabel('Date')
axes[1].set_title('Rolling Prediction Error Rate')
axes[1].legend()

plt.tight_layout()
plt.show()

---

## 4. Adaptive Learning with Drift Response

When drift is detected, we can:
1. **Reset the model** - Start fresh
2. **Increase learning rate** - Adapt faster to new patterns
3. **Use ensemble methods** - Combine old and new models
4. **Window-based retraining** - Retrain on recent data only

In [ ]:
class AdaptiveOnlineLearner:
    """
    Online learner that adapts learning rate when drift is detected.
    """
    def __init__(self, base_lr=0.01, drift_lr=0.1, cooldown=50):
        self.base_lr = base_lr
        self.drift_lr = drift_lr
        self.cooldown = cooldown
        self.steps_since_drift = cooldown
        
        self.drift_detector = ADWIN(delta=0.002)
        self._build_model(base_lr)
        
        self.metric = metrics.Accuracy()
        self.drift_events = []
        
    def _build_model(self, lr):
        self.model = compose.Pipeline(
            preprocessing.StandardScaler(),
            linear_model.LogisticRegression(optimizer=optim.SGD(lr=lr))
        )
    
    def _get_current_lr(self):
        if self.steps_since_drift < self.cooldown:
            # Decay from drift_lr to base_lr
            progress = self.steps_since_drift / self.cooldown
            return self.drift_lr * (1 - progress) + self.base_lr * progress
        return self.base_lr
    
    def predict_one(self, x):
        return self.model.predict_one(x)
    
    def learn_one(self, x, y, timestamp=None):
        # Get prediction for drift detection
        y_pred = self.model.predict_one(x)
        
        if y_pred is not None:
            error = int(y_pred != y)
            self.drift_detector.update(error)
            self.metric.update(y, y_pred)
            
            if self.drift_detector.drift_detected:
                self.steps_since_drift = 0
                self.drift_events.append(timestamp)
                # Rebuild model with higher learning rate
                self._build_model(self.drift_lr)
        
        # Learn with current model
        self.model.learn_one(x, y)
        self.steps_since_drift += 1
        
        return self


# Train adaptive model
adaptive_model = AdaptiveOnlineLearner(base_lr=0.01, drift_lr=0.1, cooldown=100)

adaptive_results = {
    'date': [],
    'prediction': [],
    'actual': [],
    'accuracy': []
}

for idx, row in X.iterrows():
    x_dict = row.to_dict()
    y_true = int(y.loc[idx])
    
    y_pred = adaptive_model.predict_one(x_dict)
    adaptive_model.learn_one(x_dict, y_true, timestamp=idx)
    
    if y_pred is not None:
        adaptive_results['date'].append(idx)
        adaptive_results['prediction'].append(y_pred)
        adaptive_results['actual'].append(y_true)
        adaptive_results['accuracy'].append(adaptive_model.metric.get())

adaptive_df = pd.DataFrame(adaptive_results).set_index('date')
print(f"Adaptive Model Final Accuracy: {adaptive_model.metric.get():.4f}")
print(f"Number of drift events: {len(adaptive_model.drift_events)}")

In [ ]:
# Compare base vs adaptive model
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(results_df.index, results_df['accuracy'], 
        label='Base Online Model', alpha=0.8)
ax.plot(adaptive_df.index, adaptive_df['accuracy'], 
        label='Adaptive Model', alpha=0.8)
ax.axhline(y=0.5, color='r', linestyle='--', label='Random Baseline', alpha=0.5)

# Mark drift events
for event in adaptive_model.drift_events:
    ax.axvline(x=event, color='orange', alpha=0.3, linewidth=1)

ax.set_ylabel('Cumulative Accuracy')
ax.set_xlabel('Date')
ax.set_title('Base vs Adaptive Online Learning Model')
ax.legend()
plt.tight_layout()
plt.show()

---

## 5. Ensemble Methods for Online Learning

Ensemble methods can improve robustness in changing environments:
- **Bagging**: Multiple models trained on different bootstrap samples
- **Adaptive Random Forest**: Random forest with drift detection per tree
- **Leveraging Bagging**: Poisson(λ) weighted bagging

In [ ]:
from river.ensemble import ADWINBaggingClassifier, LeveragingBaggingClassifier
from river.tree import HoeffdingTreeClassifier

# Create ensemble models
bagging_model = ADWINBaggingClassifier(
    model=linear_model.LogisticRegression(),
    n_models=10,
    seed=42
)

leverage_model = LeveragingBaggingClassifier(
    model=HoeffdingTreeClassifier(),
    n_models=10,
    seed=42
)

# Train and evaluate ensembles
ensemble_metrics = {
    'ADWIN Bagging': metrics.Accuracy(),
    'Leveraging Bagging': metrics.Accuracy()
}

ensemble_results = {
    'date': [],
    'ADWIN Bagging': [],
    'Leveraging Bagging': []
}

for idx, row in X.iterrows():
    x_dict = row.to_dict()
    y_true = int(y.loc[idx])
    
    # ADWIN Bagging
    pred_bagging = bagging_model.predict_one(x_dict)
    if pred_bagging is not None:
        ensemble_metrics['ADWIN Bagging'].update(y_true, pred_bagging)
    bagging_model.learn_one(x_dict, y_true)
    
    # Leveraging Bagging
    pred_leverage = leverage_model.predict_one(x_dict)
    if pred_leverage is not None:
        ensemble_metrics['Leveraging Bagging'].update(y_true, pred_leverage)
    leverage_model.learn_one(x_dict, y_true)
    
    if pred_bagging is not None and pred_leverage is not None:
        ensemble_results['date'].append(idx)
        ensemble_results['ADWIN Bagging'].append(ensemble_metrics['ADWIN Bagging'].get())
        ensemble_results['Leveraging Bagging'].append(ensemble_metrics['Leveraging Bagging'].get())

ensemble_df = pd.DataFrame(ensemble_results).set_index('date')

print("Ensemble Model Final Accuracies:")
for name, metric in ensemble_metrics.items():
    print(f"  {name}: {metric.get():.4f}")

In [ ]:
# Compare all models
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(results_df.index, results_df['accuracy'], 
        label='Single Logistic Regression', alpha=0.7)
ax.plot(adaptive_df.index, adaptive_df['accuracy'], 
        label='Adaptive Model', alpha=0.7)
ax.plot(ensemble_df.index, ensemble_df['ADWIN Bagging'], 
        label='ADWIN Bagging', alpha=0.7)
ax.plot(ensemble_df.index, ensemble_df['Leveraging Bagging'], 
        label='Leveraging Bagging', alpha=0.7)
ax.axhline(y=0.5, color='r', linestyle='--', label='Random', alpha=0.5)

ax.set_ylabel('Cumulative Accuracy')
ax.set_xlabel('Date')
ax.set_title('Online Learning Model Comparison')
ax.legend(loc='lower right')
plt.tight_layout()
plt.show()

---

## 6. Scikit-learn Incremental Learning

For compatibility with existing sklearn pipelines, use `partial_fit()` method:
- `SGDClassifier` / `SGDRegressor`
- `PassiveAggressiveClassifier`
- `MiniBatchKMeans`
- `MultinomialNB`

In [ ]:
from sklearn.linear_model import SGDClassifier, PassiveAggressiveClassifier
from sklearn.preprocessing import StandardScaler

class SklearnOnlineModel:
    """
    Wrapper for sklearn incremental learning models.
    """
    def __init__(self, model_type='sgd', warm_start_samples=100):
        self.model_type = model_type
        self.warm_start_samples = warm_start_samples
        self.scaler = StandardScaler()
        self.fitted = False
        self.buffer_X = []
        self.buffer_y = []
        
        if model_type == 'sgd':
            self.model = SGDClassifier(
                loss='log_loss',
                learning_rate='adaptive',
                eta0=0.01,
                random_state=42
            )
        elif model_type == 'pa':
            self.model = PassiveAggressiveClassifier(
                C=0.1,
                random_state=42
            )
    
    def partial_fit(self, X, y):
        if not self.fitted:
            # Buffer samples for initial fit
            self.buffer_X.append(X)
            self.buffer_y.append(y)
            
            if len(self.buffer_X) >= self.warm_start_samples:
                X_init = np.vstack(self.buffer_X)
                y_init = np.array(self.buffer_y)
                
                self.scaler.fit(X_init)
                X_scaled = self.scaler.transform(X_init)
                self.model.partial_fit(X_scaled, y_init, classes=[0, 1])
                self.fitted = True
                self.buffer_X = []
                self.buffer_y = []
        else:
            X_scaled = self.scaler.transform(X.reshape(1, -1))
            self.model.partial_fit(X_scaled, [y])
        
        return self
    
    def predict(self, X):
        if not self.fitted:
            return None
        X_scaled = self.scaler.transform(X.reshape(1, -1))
        return self.model.predict(X_scaled)[0]
    
    def predict_proba(self, X):
        if not self.fitted or not hasattr(self.model, 'predict_proba'):
            return None
        X_scaled = self.scaler.transform(X.reshape(1, -1))
        return self.model.predict_proba(X_scaled)[0]


# Train sklearn online models
sklearn_models = {
    'SGD': SklearnOnlineModel('sgd'),
    'PassiveAggressive': SklearnOnlineModel('pa')
}

sklearn_results = {name: [] for name in sklearn_models.keys()}
sklearn_dates = []

X_array = X.values
y_array = y.values

for i in range(len(X_array)):
    predictions = {}
    
    for name, model in sklearn_models.items():
        pred = model.predict(X_array[i])
        model.partial_fit(X_array[i], y_array[i])
        predictions[name] = pred
    
    if all(p is not None for p in predictions.values()):
        sklearn_dates.append(X.index[i])
        for name, pred in predictions.items():
            sklearn_results[name].append(int(pred == y_array[i]))

# Calculate cumulative accuracies
sklearn_df = pd.DataFrame(index=sklearn_dates)
for name, results in sklearn_results.items():
    sklearn_df[name] = np.cumsum(results) / np.arange(1, len(results) + 1)

print("\nSklearn Online Models Final Accuracies:")
for name in sklearn_results.keys():
    print(f"  {name}: {sklearn_df[name].iloc[-1]:.4f}")

---

## 7. Online Learning for Return Prediction (Regression)

Predicting actual returns instead of direction:

In [ ]:
from river.metrics import MAE, RMSE, R2

# Online regression model
reg_model = compose.Pipeline(
    preprocessing.StandardScaler(),
    linear_model.LinearRegression(optimizer=optim.SGD(lr=0.001))
)

# Target: next day returns
y_reg = features_df['target_return']

# Metrics
mae_metric = MAE()
rmse_metric = RMSE()
r2_metric = R2()

reg_results = {
    'date': [],
    'prediction': [],
    'actual': [],
    'mae': [],
    'rmse': []
}

for idx, row in X.iterrows():
    x_dict = row.to_dict()
    y_true = float(y_reg.loc[idx])
    
    y_pred = reg_model.predict_one(x_dict)
    
    if y_pred is not None:
        mae_metric.update(y_true, y_pred)
        rmse_metric.update(y_true, y_pred)
        r2_metric.update(y_true, y_pred)
        
        reg_results['date'].append(idx)
        reg_results['prediction'].append(y_pred)
        reg_results['actual'].append(y_true)
        reg_results['mae'].append(mae_metric.get())
        reg_results['rmse'].append(rmse_metric.get())
    
    reg_model.learn_one(x_dict, y_true)

reg_df = pd.DataFrame(reg_results).set_index('date')

print(f"Online Regression Results:")
print(f"  MAE: {mae_metric.get():.6f}")
print(f"  RMSE: {rmse_metric.get():.6f}")
print(f"  R²: {r2_metric.get():.6f}")

In [ ]:
# Visualize regression predictions
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Actual vs Predicted scatter
axes[0, 0].scatter(reg_df['actual'], reg_df['prediction'], alpha=0.3, s=5)
axes[0, 0].plot([-0.1, 0.1], [-0.1, 0.1], 'r--', label='Perfect Prediction')
axes[0, 0].set_xlabel('Actual Return')
axes[0, 0].set_ylabel('Predicted Return')
axes[0, 0].set_title('Actual vs Predicted Returns')
axes[0, 0].legend()

# MAE over time
axes[0, 1].plot(reg_df.index, reg_df['mae'])
axes[0, 1].set_ylabel('MAE')
axes[0, 1].set_title('Cumulative MAE Over Time')

# Prediction time series (last 100 days)
last_100 = reg_df.tail(100)
axes[1, 0].plot(last_100.index, last_100['actual'], label='Actual', alpha=0.7)
axes[1, 0].plot(last_100.index, last_100['prediction'], label='Predicted', alpha=0.7)
axes[1, 0].set_ylabel('Return')
axes[1, 0].set_title('Last 100 Days: Actual vs Predicted')
axes[1, 0].legend()
axes[1, 0].tick_params(axis='x', rotation=45)

# Prediction error distribution
errors = reg_df['actual'] - reg_df['prediction']
axes[1, 1].hist(errors, bins=50, edgecolor='black', alpha=0.7)
axes[1, 1].axvline(x=0, color='r', linestyle='--')
axes[1, 1].set_xlabel('Prediction Error')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Prediction Error Distribution')

plt.tight_layout()
plt.show()

---

## 8. Trading Strategy with Online Learning

Let's build a complete trading strategy using online learning:

In [ ]:
class OnlineTradingStrategy:
    """
    Trading strategy using online learning with adaptive learning rates.
    """
    def __init__(self, threshold=0.55, transaction_cost=0.001):
        self.threshold = threshold
        self.transaction_cost = transaction_cost
        
        # Model with probability output
        self.model = compose.Pipeline(
            preprocessing.StandardScaler(),
            linear_model.LogisticRegression(optimizer=optim.SGD(lr=0.01))
        )
        
        self.drift_detector = ADWIN(delta=0.002)
        self.position = 0  # 0 = flat, 1 = long, -1 = short
        
        # Track performance
        self.trades = []
        self.equity_curve = [1.0]
        self.dates = []
        
    def get_signal(self, x):
        """Generate trading signal based on prediction probability."""
        proba = self.model.predict_proba_one(x)
        
        if proba is None:
            return 0
        
        prob_up = proba.get(1, 0.5)
        
        if prob_up > self.threshold:
            return 1  # Long
        elif prob_up < (1 - self.threshold):
            return -1  # Short
        else:
            return 0  # Flat
    
    def step(self, x, y_true, actual_return, date):
        """Process one time step."""
        # Get signal before learning
        signal = self.get_signal(x)
        
        # Calculate strategy return
        position_return = self.position * actual_return
        
        # Transaction costs
        if signal != self.position:
            position_return -= self.transaction_cost * abs(signal - self.position)
            self.trades.append({
                'date': date,
                'old_position': self.position,
                'new_position': signal
            })
        
        # Update equity
        new_equity = self.equity_curve[-1] * (1 + position_return)
        self.equity_curve.append(new_equity)
        self.dates.append(date)
        
        # Update position
        self.position = signal
        
        # Learn from observation
        y_pred = self.model.predict_one(x)
        if y_pred is not None:
            self.drift_detector.update(int(y_pred != y_true))
        self.model.learn_one(x, y_true)
        
        return position_return
    
    def get_performance_metrics(self, risk_free_rate=0.0):
        """Calculate strategy performance metrics."""
        returns = pd.Series(self.equity_curve).pct_change().dropna()
        
        total_return = (self.equity_curve[-1] / self.equity_curve[0]) - 1
        annual_return = (1 + total_return) ** (252 / len(returns)) - 1
        volatility = returns.std() * np.sqrt(252)
        sharpe = (annual_return - risk_free_rate) / volatility if volatility > 0 else 0
        
        # Max drawdown
        equity = pd.Series(self.equity_curve)
        rolling_max = equity.expanding().max()
        drawdowns = equity / rolling_max - 1
        max_dd = drawdowns.min()
        
        return {
            'Total Return': f"{total_return:.2%}",
            'Annual Return': f"{annual_return:.2%}",
            'Volatility': f"{volatility:.2%}",
            'Sharpe Ratio': f"{sharpe:.2f}",
            'Max Drawdown': f"{max_dd:.2%}",
            'Num Trades': len(self.trades)
        }


# Run strategy
strategy = OnlineTradingStrategy(threshold=0.52, transaction_cost=0.0005)

for idx, row in X.iterrows():
    x_dict = row.to_dict()
    y_true = int(y.loc[idx])
    actual_return = float(y_reg.loc[idx])
    
    strategy.step(x_dict, y_true, actual_return, idx)

# Display results
print("\n=== Online Learning Strategy Performance ===")
for metric, value in strategy.get_performance_metrics().items():
    print(f"  {metric}: {value}")

In [ ]:
# Compare strategy vs buy-and-hold
buy_hold_equity = (1 + features_df['target_return']).cumprod()

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Equity curves
strategy_equity = pd.Series(strategy.equity_curve[1:], index=strategy.dates)
axes[0].plot(strategy_equity.index, strategy_equity, label='Online Learning Strategy', linewidth=1.5)
axes[0].plot(buy_hold_equity.index, buy_hold_equity, label='Buy & Hold SPY', linewidth=1.5, alpha=0.7)
axes[0].set_ylabel('Equity')
axes[0].set_title('Strategy Performance: Online Learning vs Buy & Hold')
axes[0].legend()
axes[0].set_yscale('log')

# Drawdown
strategy_dd = strategy_equity / strategy_equity.expanding().max() - 1
bh_dd = buy_hold_equity / buy_hold_equity.expanding().max() - 1

axes[1].fill_between(strategy_dd.index, strategy_dd, 0, alpha=0.5, label='Strategy Drawdown')
axes[1].fill_between(bh_dd.index, bh_dd, 0, alpha=0.5, label='Buy & Hold Drawdown')
axes[1].set_ylabel('Drawdown')
axes[1].set_xlabel('Date')
axes[1].set_title('Drawdown Comparison')
axes[1].legend()

plt.tight_layout()
plt.show()

---

## 9. Best Practices for Online Learning in Production

### Key Considerations:

1. **Data Quality**
   - Handle missing values gracefully
   - Detect and handle outliers online
   - Validate feature distributions

2. **Model Stability**
   - Use regularization to prevent overfitting to noise
   - Implement learning rate schedules
   - Monitor for catastrophic forgetting

3. **Drift Management**
   - Multiple drift detectors for robustness
   - Gradual vs sudden drift handling
   - Maintain historical model checkpoints

4. **Monitoring**
   - Track prediction accuracy over time
   - Monitor feature importance changes
   - Alert on performance degradation

In [ ]:
class ProductionOnlineLearner:
    """
    Production-ready online learning system with monitoring.
    """
    def __init__(self, model, drift_threshold=0.002, window_size=100):
        self.model = model
        self.drift_detector = ADWIN(delta=drift_threshold)
        self.window_size = window_size
        
        # Monitoring
        self.predictions = []
        self.actuals = []
        self.timestamps = []
        self.drift_events = []
        self.feature_stats = {}
        
        # Performance tracking
        self.rolling_accuracy = []
        
    def predict(self, x, timestamp=None):
        """Make prediction with monitoring."""
        pred = self.model.predict_one(x)
        
        # Update feature statistics
        for feature, value in x.items():
            if feature not in self.feature_stats:
                self.feature_stats[feature] = {'values': []}
            self.feature_stats[feature]['values'].append(value)
            # Keep only recent values
            if len(self.feature_stats[feature]['values']) > self.window_size * 10:
                self.feature_stats[feature]['values'] = \
                    self.feature_stats[feature]['values'][-self.window_size * 5:]
        
        return pred
    
    def learn(self, x, y_true, timestamp=None):
        """Learn from observation with drift detection."""
        y_pred = self.model.predict_one(x)
        
        if y_pred is not None:
            # Track predictions
            self.predictions.append(y_pred)
            self.actuals.append(y_true)
            self.timestamps.append(timestamp)
            
            # Drift detection
            error = int(y_pred != y_true)
            self.drift_detector.update(error)
            
            if self.drift_detector.drift_detected:
                self.drift_events.append({
                    'timestamp': timestamp,
                    'rolling_acc': self.get_rolling_accuracy()
                })
            
            # Update rolling accuracy
            self.rolling_accuracy.append(self.get_rolling_accuracy())
        
        self.model.learn_one(x, y_true)
        return self
    
    def get_rolling_accuracy(self):
        """Calculate rolling accuracy."""
        if len(self.predictions) < self.window_size:
            return None
        recent_preds = self.predictions[-self.window_size:]
        recent_actuals = self.actuals[-self.window_size:]
        return np.mean([p == a for p, a in zip(recent_preds, recent_actuals)])
    
    def get_feature_drift_report(self):
        """Check for feature distribution drift."""
        report = {}
        for feature, stats in self.feature_stats.items():
            values = stats['values']
            if len(values) > self.window_size * 2:
                old_values = values[:self.window_size]
                new_values = values[-self.window_size:]
                
                mean_shift = abs(np.mean(new_values) - np.mean(old_values)) / (np.std(old_values) + 1e-8)
                std_ratio = np.std(new_values) / (np.std(old_values) + 1e-8)
                
                report[feature] = {
                    'mean_shift_zscore': mean_shift,
                    'std_ratio': std_ratio,
                    'drift_suspected': mean_shift > 2 or std_ratio > 2 or std_ratio < 0.5
                }
        return report
    
    def get_monitoring_summary(self):
        """Get monitoring summary."""
        if not self.predictions:
            return "No predictions made yet."
        
        overall_acc = np.mean([p == a for p, a in zip(self.predictions, self.actuals)])
        
        return {
            'total_samples': len(self.predictions),
            'overall_accuracy': f"{overall_acc:.4f}",
            'rolling_accuracy': f"{self.get_rolling_accuracy():.4f}" if self.get_rolling_accuracy() else "N/A",
            'drift_events': len(self.drift_events),
            'features_with_drift': sum(1 for f, r in self.get_feature_drift_report().items() 
                                       if r.get('drift_suspected', False))
        }


# Demo production system
prod_model = compose.Pipeline(
    preprocessing.StandardScaler(),
    linear_model.LogisticRegression(optimizer=optim.SGD(lr=0.01))
)

prod_learner = ProductionOnlineLearner(prod_model, window_size=100)

# Simulate production environment
for idx, row in X.iterrows():
    x_dict = row.to_dict()
    y_true = int(y.loc[idx])
    
    pred = prod_learner.predict(x_dict, timestamp=idx)
    prod_learner.learn(x_dict, y_true, timestamp=idx)

# Get monitoring report
print("\n=== Production Monitoring Summary ===")
summary = prod_learner.get_monitoring_summary()
for key, value in summary.items():
    print(f"  {key}: {value}")

print("\n=== Feature Drift Report ===")
drift_report = prod_learner.get_feature_drift_report()
drifted_features = [f for f, r in drift_report.items() if r.get('drift_suspected', False)]
print(f"  Features with suspected drift: {drifted_features if drifted_features else 'None'}")

---

## 10. Summary & Key Takeaways

### Online Learning Advantages for Trading:
1. **Adaptability**: Models continuously adapt to new market regimes
2. **Efficiency**: No need to store/reprocess all historical data
3. **Real-time**: Updates as new data arrives
4. **Memory**: Constant memory usage regardless of data size

### Key Algorithms Covered:
- **River**: LogisticRegression, LinearRegression, Hoeffding Trees
- **Sklearn**: SGDClassifier, PassiveAggressiveClassifier
- **Drift Detection**: ADWIN, DDM, Page-Hinkley
- **Ensembles**: ADWIN Bagging, Leveraging Bagging

### Production Considerations:
- Monitor model performance continuously
- Implement drift detection and response
- Track feature distributions for data quality
- Maintain model checkpoints for rollback

### Challenges:
- Lower accuracy than batch models with full data
- Sensitivity to learning rate tuning
- Risk of overfitting to recent noise
- Need for robust monitoring infrastructure

In [ ]:
# Final comparison table
comparison_data = {
    'Model': ['River Logistic', 'Adaptive Model', 'ADWIN Bagging', 
              'Leveraging Bagging', 'SKLearn SGD', 'SKLearn PA'],
    'Final Accuracy': [
        results_df['accuracy'].iloc[-1],
        adaptive_df['accuracy'].iloc[-1],
        ensemble_df['ADWIN Bagging'].iloc[-1],
        ensemble_df['Leveraging Bagging'].iloc[-1],
        sklearn_df['SGD'].iloc[-1],
        sklearn_df['PassiveAggressive'].iloc[-1]
    ]
}

comparison_table = pd.DataFrame(comparison_data)
comparison_table['Final Accuracy'] = comparison_table['Final Accuracy'].apply(lambda x: f"{x:.4f}")
comparison_table = comparison_table.sort_values('Final Accuracy', ascending=False)

print("\n=== Online Learning Model Comparison ===")
print(comparison_table.to_string(index=False))

---

## Exercises

1. **Multi-Asset Online Learning**: Extend the strategy to trade multiple assets simultaneously

2. **Feature Selection**: Implement online feature importance tracking and dynamic feature selection

3. **Regime Detection**: Build a regime detection system that adjusts model hyperparameters based on detected market regime

4. **Ensemble Strategies**: Create an ensemble that combines batch-trained and online models

5. **Latency Analysis**: Measure and optimize the prediction latency of online models for HFT applications

---

## References

1. River Documentation: https://riverml.xyz/
2. Gama, J., et al. (2014). "A Survey on Concept Drift Adaptation"
3. Bifet, A., & Gavaldà, R. (2007). "Learning from Time-Changing Data with Adaptive Windowing"
4. Shalev-Shwartz, S. (2012). "Online Learning and Online Convex Optimization"